# Detect ARs from individual time steps


This notebook detects ARs at instantaneous time steps.

The detection is performed mainly on the anomalous IVT computed in the previous step (in notebook `2 compute_THR`), using these steps:

1. At any time point, find all contiguous regions from the `ivt_ano` field where `ivt_ano` $\ge 0\, kg/m/s$.
2. Compute the centroid of all such regions, using the underlying IVT values as weights.
3. Discard all regions whose area is $\le 50 \times 10^4\, km^2$, or $\ge 1800 \times 10^4\,km^2$.
4. Discard all regions whose centroid lies north of $80^{\circ}\, N$, or south of $20^{\circ} N$.
5. Discard all regions whose  [isoperimeteric quotient](https://en.wikipedia.org/wiki/Isoperimetric_inequality) ($q = \frac{4 \pi A}{L^2} \ge 0.6$) This is to filter out circular features like tropical cyclones.
6. Compute the AR axis.
7. Discard all regions whose AR axis is $\le 1500\, km$.
8. Compute the effective width as area/length, and the length/width ratio.
9. Discard all regions whose length/width ratio is $\le 2$.

All passing systems after the above steps are regarded as ARs.

There are some more details given in the parameter selection section down below.

In production, you can use the `scripts/detect_ARs.py` or `scripts/detect_ARs_generator_version.py` for this step.


## Input data

* `uflux_m1-60_6_1984_crop.nc`: u-component of vertically integrated vapor fluxes, in kg/m/s.
* `vflux_m1-60_6_1984_crop.nc`: v-component of vertically integrated vapor fluxes, in kg/m/s.
* `ivt_m1-60_6_1984_crop-THR-kernel-t16-s6.nc`: decomposition of IVT using the THR algorithm, into a reconstruction component (`ivt_rec`) and the anomaly component (`ivt_ano`), in kg/m/s.


## Steps

1. Make sure you have successfully run the previous notebook.
2. Execute the following code blocks in sequence.


## Results


* `ar_records.csv`: a csv table listing various attributes for each detected AR appearance.
* `ar_s_6_1984_labels.nc`: a netCDF data file saving these 3 variables:
    * `label`: an integer label distinguishing all ARs detected at individual time steps.
    * `angles`: the angle between horizontal vapor flux and the local AR axis, in degrees.
    * `ivt_cross`: cross-sectional IVT flux, computed as the product of IVT vector and cosine of `angles`.
* `plots/ar_1984-01-02 18:00.png` (optional): plots of IVT and detected ARs at individual time steps.

All results will be saved to the same folder as this notebook file.

## Set some global parameters

`YEAR`, `TIME_START`, and `TIME_END` are used to specify the time domain to process.

In this notebook only a small time subset is specified. In production you would want to set the time points to suit your needs.

In [ ]:
#--------------------Time range--------------------
YEAR=1984
TIME_START='%d-03-01 00:00:00' %YEAR
TIME_END='%d-03-09 18:00:00' %YEAR

Specify the input and output locations. A subfolder is created using the `YEAR` defined above.

In [ ]:
%matplotlib inline
import os, sys

#-----------u-qflux----------------------
UQ_FILE_NAME=os.path.join('.', 'uflux_m1-60_6_1984_crop.nc')
UQ_VAR='uflux'

#-----------v-qflux----------------------
VQ_FILE_NAME=os.path.join('.', 'vflux_m1-60_6_1984_crop.nc')
VQ_VAR='vflux'

#-----------------ivt reconstruction and anomalies-----------------
IVT_FILE_NAME=os.path.join('.', 'ivt_m1-60_6_1984_crop-THR-kernel-t16-s6.nc')

#------------------Output folder------------------
OUTPUTDIR=os.path.join('.', str(YEAR))

* `PLOT` controls whether to plot figures of IVT with detected ARs.
* `LAT1` and `LAT2` selects the latitude range to process.
* `RESO` is the (approximate) horizontal resolution of input data. For the ERA-I data used in this notebook it is 0.75.
* `SHIFT_LON` shifts the data along the x-dimension by 80 degrees so the Pacific and Atlantic oceans are centered.


In [ ]:
PLOT=True          # create maps of found ARs or not

LAT1=0; LAT2=90      # degree, latitude domain
# NOTE: this has to match the domain selection in compute_thr_singlefile.py

RESO=0.75             # degree, (approximate) horizontal resolution of input data.
SHIFT_LON=80          # degree, shift left bound to longitude. Should match
                      # that used in compute_thr_singlefile.py

The `PARAM_DICT` dictionary contains important parameters used in the detection process:

* `thres_low`: float, a minimum IVT value in $kg/m/s$. This tells the script to look for AR candidates as regions where the anomaly IVT (`ivt_ano`) >= this value. This is the same idea as the IVT250 thresholding method. In production one should give it a small nominal value like `1`, `10` etc., or just `0`.
* `min_area`: float, minimum area in $km^2$. Drop AR candidates smaller than this area. Region of area is defined as the summation of grid cell areas, computed using the latitude/longitude meta data of input data. This is used to filter out some miniature features.
* `max_area`: float, maximum area in $km^2$. Filter out regions too large in size. This might happen when 2 AR-like features get merged together. You can prevent this from happening by raising the `thres_low` value, or setting `SINGLE_DOME` to true.
* `max_isoq_hard`: float in (0,1), the maximum isoperimeteric quotient. This is to filter out circular features like tropical cyclones.
* `max_isoq`: AR candidates with isoperimeteric quotient larger than this values is flagged as `relaxed`, meaning that they may not be a typical AR, but kind of close. This effectively serves as a simple fuzzy logic control in the AR detection.
* `min_lat`: float, degree North, exclude systems whose centroids are lower than this latitude.
* `max_lat`: float, degree North, exclude systems whose centroids are higher than this latitude.
* `min_length`: float, km, AR candidates shorter than this length are flagged as `relaxed`.
* `min_length_hard`: float, km, AR candidates shorter than this length are discarded.
* `rdp_thres`: float, degree latitude/longitude, the user given error when simplifying axis using [rdp algorithm](https://en.wikipedia.org/wiki/Ramer%E2%80%93Douglas%E2%80%93Peucker_algorithm).
* `fill_radius`: this is to fill up some holes found in the AR candidate regions. This can happen when your input data have pretty high resolution and tend to have more small scale features in the IVT field. Sometimes this will leave some small holes in the found AR region.
* `single_dome`: `True` or `False` values control whether to separate local IVT maxima that are merged into one contour.
* `max_ph_ratio`: float in (0,1). Maximum prominence/height ratio of a local peak. Only used when SINGLE_DOME=True.
* `edge_eps`: float in (0,1). Minimal proportion of flux component in a direction to total flux to
    allow edge building in that direction. Setting this to a higher value will impose greater restriction upon the directions of the AR axis, requiring it to more strictly follow the vectors of IVT. Setting a lower value gives more maneuver space of the AR axis to pass through the AR region.

In [ ]:
PARAM_DICT={
    # kg/m/s, define AR candidates as regions >= than this anomalous ivt.
    'thres_low' : 1,
    # km^2, drop AR candidates smaller than this area.
    'min_area': 50*1e4,
    # km^2, drop AR candidates larger than this area.
    'max_area': 1800*1e4,
    # float, isoperimetric quotient. ARs larger than this (more circular in shape) is treated as relaxed.
    'max_isoq': 0.6,
    # float, isoperimetric quotient. ARs larger than this is discarded.
    'max_isoq_hard': 0.7,
    # degree, exclude systems whose centroids are lower than this latitude.
    'min_lat': 20,
    # degree, exclude systems whose centroids are higher than this latitude.
    'max_lat': 80,
    # km, ARs shorter than this length is treated as relaxed.
    'min_length': 2000,
    # km, ARs shorter than this length is discarded.
    'min_length_hard': 1500,
    # degree lat/lon, error when simplifying axis using rdp algorithm.
    'rdp_thres': 2,
    # grids. Remove small holes in AR contour.
    'fill_radius': max(1,int(4*0.75/RESO)),
    # do peak partition or not, used to separate systems that are merged
    # together with an outer contour.
    'single_dome': False,
    # max prominence/height ratio of a local peak. Only used when single_dome=True
    'max_ph_ratio': 0.6,
    # minimal proportion of flux component in a direction to total flux to
    # allow edge building in that direction
    'edge_eps': 0.4
    }

Import the modules.

In [ ]:
#--------Import modules-------------------------
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cdms2 as cdms

from ipart.utils import funcs,plot
from ipart.AR_detector import plotAR, findARs

Then read in the input data, and do some slicing to select the part of data we want:

In [ ]:
#-----------Read in flux data----------------------
qu=funcs.readVar(UQ_FILE_NAME, UQ_VAR)
qv=funcs.readVar(VQ_FILE_NAME, VQ_VAR)

#-----------------Shift longitude-----------------
qu=qu(longitude=(SHIFT_LON,SHIFT_LON+360))
qv=qv(longitude=(SHIFT_LON,SHIFT_LON+360))

#-------------------Read in ivt-------------------
print('\n### <river_tracker1>: Read in file:\n',IVT_FILE_NAME)
fin=cdms.open(IVT_FILE_NAME,'r')
ivt=fin('ivt')
ivtrec=fin('ivt_rec')
ivtano=fin('ivt_ano')
fin.close()

#--------------------Slice data--------------------
qu=qu(time=(TIME_START,TIME_END), latitude=(LAT1, LAT2))(squeeze=1)
qv=qv(time=(TIME_START,TIME_END), latitude=(LAT1, LAT2))(squeeze=1)
ivt=ivt(time=(TIME_START,TIME_END))(squeeze=1)
ivtrec=ivtrec(time=(TIME_START,TIME_END))(squeeze=1)
ivtano=ivtano(time=(TIME_START,TIME_END))(squeeze=1)

We then fetch the time, latitude/longitude axes from the data. The time axis is assumed to be a list of strings, e.g. `['1984-01-01 00:00', '1984-01-01 06:00', ..., '1984-05-31 00:00']`. The latitude/longitude axis is assumed to be a 1d array storing the latitude/longitude coordinates in degrees of lat/lon.

In this script we are using the `CDAT` (aka `UVCDAT`, `CDAT8`) to fetch these metadata from the netCDF variables. If you are using other packages like `netcdf4`, `xarray` or `iris`, please adjust the relevant codes accordingly.

In [ ]:
#-----------------Get coordinates-----------------
latax=qu.getLatitude()
lonax=qu.getLongitude()
timeax=ivt.getTime().asComponentTime()
timeax=['%d-%02d-%02d %02d:00' %(timett.year, timett.month, timett.day, timett.hour) for timett in timeax]

All prepared, we can now call the detection function `findARs()` that does most of the heavy lifting work, including:

* detect candidate regions satisfying requirements given in the `PARAM_DICT` dict.
* for each passing candidate, compute an AR axis.
* fetches some information from each AR, including:
    * its numerical label,
    * length,
    * area,
    * width (defined as area/length),
    * centroid coordinates,
    * axis coordinates,
    * contour coordinates,
    * average IVT strength,
    * others.

These information is saved in a `pandas.DataFrame` named `result_df`.

`findARs()` also outputs these variables:

* `time_idx`: a list of indices of the time dimension when any AR is found.
* `labels` is a netcdf variable saving the numerical labels of all found ARs in each time step. It has shape of `(time, lat, lon)`.
* `angles` is a netcdf variable saving the difference in the orientation of IVT vectors in all found ARs, wrt the AR axis. It is not relevant at this stage.
* `crossfluxes` is a netcdf variable saving the cross-sectional IVT flux, computed as the projection of IVT vectors onto the AR axis, using angles in `angles`. It is not relevant at this stage.

In [ ]:
time_idx, labels, angles, crossfluxes, result_df = findARs(ivt, ivtrec,
            ivtano, qu, qv, latax, lonax, PARAM_DICT, times=timeax)

## Show some results

After the computation is done, we could have a look at some of the results.

First print the number of ARs detected:

In [ ]:
print("Number of ARs found during %s - %s = %d." %(TIME_START, TIME_END, len(result_df)))

Then print the first few records:

In [ ]:
result_df.head(4)

The columns of this table:

* **id**: integer numeric id for this AR at *this particular* time point. ARs at different time points can share the same **id**, and an AR can be uniquely identified with the combination of time stamp + id.
* **time**: time stamp in the YYYY-MM-DD HH:mm:ss format.
* **contour_y**: list of floats, the y-coordinates (latitudes) of the AR contour in degrees North.
* **contour_x**: list of floats, the x-coordinates (longitude) of the AR contour in degrees North.
* **centroid_y**: float, latitude of the AR centroid, weighted by the IVT value.
* **centroid_x**: float, longitude of the AR centroid, weighted by the IVT value.
* **axis_y**: list of floats, latitudes of the AR axis.
* **axis_x**: list of floats, longitude of the AR axis.
* **axis_rdp_y**: list of floats, latitude of the simplified AR axis.
* **axis_rdp_x**: list of floats, longitude of the simplified AR axis.
* **area**: float, area of the AR in $km^2$.
* **length**: float, length of the AR in $km$.
* **width**: float, effective width in $km$, as area/length.
* **iso_quotient**: float, isoperimeteric quotient.
* **LW_ratio**: float, length/width ratio.
* **strength**: float, spatially averaged IVT value within the AR region, in $kg/m/s$.
* **strength_ano**: float, spatially averaged *anomalous* IVT value within the AR region, in $kg/m/s$.
* **strength_std**: float, standard deviation of IVT within the AR region, in $kg/m/s$.
* **max_strength**: float, maximum IVT value within the AR region, in $kg/m/s$.
* **mean_angle**: float, spatially averaged angle between the IVT vector and the AR axis, in degrees.
* **is_relaxed**: True or False, whether the AR is flagged as "relaxed".
* **qv_mean**: float, spatially averaged meridional integrated vapor flux, in $kg/m/s$.




Now create some plots for the last time step with any detected ARs.

In [ ]:
plot_idx=time_idx[-1]

plot_time=timeax[plot_idx]
slab=ivt[plot_idx]
slabrec=ivtrec[plot_idx]
slabano=ivtano[plot_idx]
ardf=result_df[result_df.time==plot_time]

plot_vars=[slab, slabrec, slabano]
titles=['IVT', 'THR_recon', 'THR_ano']
iso=plot.Isofill(plot_vars, 12, 1, 1,min_level=0,max_level=800)

figure=plt.figure(figsize=(12,10), dpi=100)

for jj in range(len(plot_vars)):
    ax=figure.add_subplot(3,1,jj+1)
    pobj=plot.plot2(plot_vars[jj], iso, ax, projection='cyl',
            title='%s %s' %(timett, titles[jj]),
            fix_aspect=False)
    bmap=pobj.bmap
    plotAR(ardf, ax, bmap)

Lastly, save the outputs to disk.

The `result_df` is saved to a `.csv` file.

The `np.set_printoptions()` function makes sure that in the saved `csv` table, the value in any cell does not contain any ellipsis `...`. This is because the coordinates of an AR axis is a list of float numbers. When this list goes too long, an ellipsis will be inserted in the saved `csv` output, e.g.

```
[12.232, 15.234, 17.3435, ..., 20.123, 24.333]
```

The same is also true for the AR contour coordinates.

To prevent this, the `np.set_printoptions()` function is called, with different input arguments for py2 and py3.

In [ ]:
if not os.path.exists(OUTPUTDIR):
    os.makedirs(OUTPUTDIR)

abpath_out=os.path.join(OUTPUTDIR, 'ar_records.csv')
print('\n### <detect_ARs>: Saving output to:\n',abpath_out)
# Necessary: to remove ... in csv file
if sys.version_info.major==2:
    np.set_printoptions(threshold=np.inf)
elif sys.version_info.major==3:
    np.set_printoptions(threshold=sys.maxsize)
result_df.to_csv(abpath_out,index=False)

Then save the labels to a `.nc` file.

In [ ]:
abpath_out=os.path.join(OUTPUTDIR, 'ar_s_6_%d_labels.nc' %YEAR)
print('\n### <detect_ARs>: Saving output to:\n',abpath_out)
ncfout=cdms.open(abpath_out,'w')
ncfout.write(labels)
ncfout.write(angles, typecode='f')
ncfout.write(crossfluxes, typecode='f')
ncfout.close()